# 01 — Connect and Explore

This notebook demonstrates connecting to a TastyTrade paper account and exploring available option chains.

## Prerequisites

1. A TastyTrade paper trading account (sign up at https://tastytrade.com)
2. Set environment variables before launching Jupyter:
   ```bash
   export TASTYTRADE_USERNAME='your_username'
   export TASTYTRADE_PASSWORD='your_password'
   ```
3. Or edit `config/config.yaml` with your credentials

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. Setup & Config

Load configuration from YAML and display settings (credentials are redacted via `SecretStr`).

In [ ]:
import os
from pathlib import Path

from options_analyzer.config.schema import AppConfig, ProviderConfig

config_path = Path("../config/config.yaml")

if config_path.exists():
    config = AppConfig.from_yaml(config_path)
    # Override with env vars if present
    username = os.environ.get("TASTYTRADE_USERNAME")
    password = os.environ.get("TASTYTRADE_PASSWORD")
    if username and password:
        config = config.model_copy(
            update={"provider": ProviderConfig(
                username=username,
                password=password,
                is_paper=config.provider.is_paper,
            )}
        )
else:
    # Build config from env vars
    config = AppConfig(
        provider=ProviderConfig(
            username=os.environ["TASTYTRADE_USERNAME"],
            password=os.environ["TASTYTRADE_PASSWORD"],
            is_paper=True,
        )
    )

# SecretStr hides credentials in repr
print(f"Provider: {config.provider.name}")
print(f"Paper trading: {config.provider.is_paper}")
print(f"Username: {config.provider.username}")

## 2. Connect to TastyTrade

Create a session and authenticate. The session uses `is_test=True` for paper trading.

In [ ]:
from options_analyzer.adapters.tastytrade.session import TastyTradeSession
from options_analyzer.adapters.tastytrade.market_data import TastyTradeMarketDataProvider
from options_analyzer.adapters.tastytrade.account import TastyTradeAccountProvider

tt_session = TastyTradeSession(config.provider)
await tt_session.connect()

market_data = TastyTradeMarketDataProvider(tt_session)
account_provider = TastyTradeAccountProvider(tt_session)

# Verify connection by listing accounts
accounts = await account_provider.get_accounts()
print(f"Connected! Accounts: {accounts}")

## 3. Fetch Option Chain

Retrieve the full option chain for a liquid underlying. The chain is organized by expiration date.

In [ ]:
UNDERLYING = "SPY"

chain = await market_data.get_option_chain(UNDERLYING)

print(f"\n{UNDERLYING} Option Chain")
print(f"{'='*40}")
print(f"Expirations available: {len(chain)}")
print()
for exp_date in sorted(chain.keys())[:10]:
    n_contracts = len(chain[exp_date])
    print(f"  {exp_date}  —  {n_contracts} contracts")

if len(chain) > 10:
    print(f"  ... and {len(chain) - 10} more expirations")

## 4. Explore Chain Data

Filter to a specific expiration, display calls and puts, and identify the ATM strike.

In [ ]:
from decimal import Decimal

# Get underlying price
spot_price = await market_data.get_underlying_price(UNDERLYING)
print(f"{UNDERLYING} current mid price: ${spot_price:.2f}")

# Pick the nearest monthly expiration (skip weeklies — pick one with many strikes)
sorted_exps = sorted(chain.keys())
target_exp = max(sorted_exps[:5], key=lambda d: len(chain[d]))
contracts = chain[target_exp]

print(f"\nSelected expiration: {target_exp}")
print(f"Total contracts: {len(contracts)}")

# Separate calls and puts
from options_analyzer.domain.enums import OptionType

calls = sorted(
    [c for c in contracts if c.option_type == OptionType.CALL],
    key=lambda c: c.strike,
)
puts = sorted(
    [c for c in contracts if c.option_type == OptionType.PUT],
    key=lambda c: c.strike,
)

print(f"Calls: {len(calls)}, Puts: {len(puts)}")

# Find ATM strike
atm_strike = min(calls, key=lambda c: abs(c.strike - spot_price)).strike
print(f"ATM strike: {atm_strike}")

## 5. Display as DataFrame

Convert option contracts near ATM to a table for easy viewing.

In [ ]:
# Show contracts near ATM (± 10 strikes)
near_atm = [
    c for c in contracts
    if abs(c.strike - atm_strike) <= Decimal("10")
]
near_atm.sort(key=lambda c: (c.strike, c.option_type))

# Build a simple table
print(f"\n{'Strike':>8}  {'Type':>5}  {'Symbol':<35}  {'Style':<10}  {'Mult':>4}")
print(f"{'─'*8}  {'─'*5}  {'─'*35}  {'─'*10}  {'─'*4}")
for c in near_atm:
    marker = " ◄ ATM" if c.strike == atm_strike else ""
    print(
        f"{c.strike:>8}  {c.option_type.value:>5}  {c.symbol:<35}  "
        f"{c.exercise_style.value:<10}  {c.multiplier:>4}{marker}"
    )

## 6. Cleanup

Disconnect the session to release resources.

In [ ]:
await tt_session.disconnect()
print("Session disconnected.")